In [1]:
%load_ext autoreload 
%autoreload 2

In [2]:
import os 
os.chdir('/home/oldrain123/MMD/')
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [10]:
import numpy as np
import torch
import pickle
from tqdm.auto import tqdm
from sklearn.utils import check_random_state
from utils_HD import MatConvert, MMDu, TST_MMD_u

In [4]:
def sample_blobs_Q(N1, sigma_mx_2, rows=3, cols=3, rs=None):
    """Generate Blob-D for testing type-II error (or test power)."""
    rs = check_random_state(rs)
    mu = np.zeros(2)
    sigma = np.eye(2) * 0.03
    X = rs.multivariate_normal(mu, sigma, size=N1)
    Y = rs.multivariate_normal(mu, np.eye(2), size=N1)
    # assign to blobs 
    X[:, 0] += rs.randint(rows, size=N1)
    X[:, 1] += rs.randint(cols, size=N1)
    Y_row = rs.randint(rows, size=N1)
    Y_col = rs.randint(cols, size=N1)
    locs = [[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[2,0],[2,1],[2,2]]
    for i in range(9):
        corr_sigma = sigma_mx_2[i]
        L = np.linalg.cholesky(corr_sigma)
        ind = np.expand_dims((Y_row == locs[i][0]) & (Y_col == locs[i][1]), 1)
        ind2 = np.concatenate((ind, ind), 1)
        Y = np.where(ind2, np.matmul(Y, L) + locs[i], Y)
    return X, Y

In [5]:
class ModelLatentF(torch.nn.Module):
    """Latent space for both domains."""

    def __init__(self, x_in, H, x_out):
        """Init latent features."""
        super(ModelLatentF, self).__init__()
        self.restored = False

        self.latent = torch.nn.Sequential(
            torch.nn.Linear(x_in, H, bias=True),
            torch.nn.Softplus(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Softplus(),
            torch.nn.Linear(H, H, bias=True),
            torch.nn.Softplus(),
            torch.nn.Linear(H, x_out, bias=True),
        )
    def forward(self, input):
        """Forward the LeNet."""
        fealant = self.latent(input)
        return fealant

In [6]:
# Setup seeds
np.random.seed(1102)
torch.manual_seed(1102)
torch.cuda.manual_seed(1102)
torch.backends.cudnn.deterministic = True
is_cuda = True

In [7]:
# Setup for experiments
dtype = torch.float
device = torch.device("cuda:0")
N_per = 100 # permutation times
alpha = 0.05 # test threshold
n_list = [10, 20, 40, 50, 70, 80, 90, 100] # number of samples in per mode
x_in = 2
H = 50 
x_out = 50
learning_rate = 0.0005
N_epoch = 1000 # number of training epochs
K = 10 # number of trails
N = 100 # number of test sets
N_f = 100.0 # number of test sets (float)

In [8]:
# Generate variance and co-variance matrix of Q 
sigma_mx_2_standard = np.array([[0.03, 0], [0, 0.03]])
sigma_mx_2 = np.zeros([9, 2, 2])
for i in range(9):
    sigma_mx_2[i] = sigma_mx_2_standard
    if i < 4:
        sigma_mx_2[i][0 ,1] = -0.02 - 0.002 * i
        sigma_mx_2[i][1, 0] = -0.02 - 0.002 * i
    if i==4:
        sigma_mx_2[i][0, 1] = 0.00
        sigma_mx_2[i][1, 0] = 0.00
    if i>4:
        sigma_mx_2[i][1, 0] = 0.02 + 0.002 * (i-5)
        sigma_mx_2[i][0, 1] = 0.02 + 0.002 * (i-5)

In [12]:
for n in n_list:
    np.random.seed(1102)
    torch.manual_seed(1102)
    torch.cuda.manual_seed(1102)
    N1 = 9 * n
    N2 = 9 * n
    Results = np.zeros([1, K])
    Opt_ep = np.zeros([1, K])
    J_star_u = np.zeros([K, N_epoch])
    ep_OPT = np.zeros([K])
    s_OPT = np.zeros([K])
    s0_OPT = np.zeros([K])
    # Repeat experiments K times (K = 10) and report average test power (rejection rate)
    for kk in tqdm(range(K),desc="Experiment"):
        # Initialize parameters
        if is_cuda:
            model_u = ModelLatentF(x_in, H, x_out).cuda()
        else:
            model_u = ModelLatentF(x_in, H, x_out)
        epsilonOPT = MatConvert(np.random.rand(1) * (10 ** (-10)), device, dtype)
        epsilonOPT.requires_grad = True
        sigmaOPT = MatConvert(np.sqrt(np.random.rand(1) * 0.3), device, dtype)
        sigmaOPT.requires_grad = True
        sigma0OPT = MatConvert(np.sqrt(np.random.rand(1) * 0.002), device, dtype)
        sigma0OPT.requires_grad = True
        # Setup optimizer for training deep kernel
        optimizer_u = torch.optim.Adam(list(model_u.parameters())+[epsilonOPT]+[sigmaOPT]+[sigma0OPT], lr=learning_rate) #
        # Generate Blob-D
        np.random.seed(seed=112 * kk + 1 + n)
        s1,s2 = sample_blobs_Q(N1, sigma_mx_2)
        # REPLACE above line with
        # s1,s2 = sample_blobs(N1)
        # for validating type-I error (s1 ans s2 are from the same distribution)
        if kk==0:
            s1_o = s1
            s2_o = s2
        S = np.concatenate((s1, s2), axis=0)
        S = MatConvert(S, device, dtype)
        # Train deep kernel to maximize test power
        np.random.seed(seed=1102)
        torch.manual_seed(1102)
        torch.cuda.manual_seed(1102)
        for t in range(N_epoch):
            # Compute epsilon, sigma and sigma_0
            ep = torch.exp(epsilonOPT)/(1+torch.exp(epsilonOPT))
            sigma = sigmaOPT ** 2
            sigma0_u = sigma0OPT ** 2
            # Compute output of the deep network
            modelu_output = model_u(S)
            # Compute J (STAT_u)
            TEMP = MMDu(modelu_output, N1, S, sigma, sigma0_u, ep, complete=True)
            mmd_value_temp = -1 * TEMP[0]
            mmd_std_temp = torch.sqrt(TEMP[1]+10**(-5))
            STAT_u = torch.div(mmd_value_temp, mmd_std_temp)
            J_star_u[kk, t] = STAT_u.item()
            # Initialize optimizer and Compute gradient
            optimizer_u.zero_grad()
            STAT_u.backward(retain_graph=True)
            # Update weights using gradient descent
            optimizer_u.step()
            # Print MMD, std of MMD and J
            if t % 100 == 0:
                print("mmd_value: ", -1 * mmd_value_temp.item(), "mmd_std: ", mmd_std_temp.item(), "Statistic J: ",
                      -1 * STAT_u.item())
        h_u, threshold_u, mmd_value_u = TST_MMD_u(model_u(S), N_per, N1, S, sigma, sigma0_u, ep, alpha, device,
                                                  dtype, complete=True)
        ep_OPT[kk] = ep.item()
        s_OPT[kk] = sigma.item()
        s0_OPT[kk] = sigma0_u.item()
        print(ep, epsilonOPT)
        # Compute test power of deep kernel based MMD
        H_u = np.zeros(N)
        T_u = np.zeros(N)
        M_u = np.zeros(N)
        np.random.seed(1102)
        count_u = 0
        for k in tqdm(range(N), desc="testing"):
            # Generate Blob-D
            np.random.seed(seed=11 * k + 10 + n)
            s1,s2 = sample_blobs_Q(N1, sigma_mx_2)
            # REPLACE above line with
            # s1,s2 = sample_blobs(N1)
            # for validating type-I error (s1 ans s2 are from the same distribution)
            S = np.concatenate((s1, s2), axis=0)
            S = MatConvert(S, device, dtype)
            # Run two sample test (deep kernel) on generated data
            h_u, threshold_u, mmd_value_u = TST_MMD_u(model_u(S), N_per, N1, S, sigma, sigma0_u, ep, alpha, device, dtype, complete=True)
            # Gather results
            count_u = count_u + h_u
            print("MMD-DK:", count_u, "Threshold:", threshold_u, "MMD_value:", mmd_value_u)
            H_u[k] = h_u
            T_u[k] = threshold_u
            M_u[k] = mmd_value_u
        # Print test power of MMD-D
        print("n =",str(n),"--- Test Power of MMD-D: ", H_u.sum()/N_f)
        Results[0, kk] = H_u.sum() / N_f
        print("n =",str(n),"--- Test Power of MMD-D (K times): ",Results[0])
        print("n =",str(n),"--- Average Test Power of MMD-D: ",Results[0].sum()/(kk+1))

Experiment:   0%|          | 0/10 [00:00<?, ?it/s]

mmd_value:  0.002211950719356537 mmd_std:  0.007882216040019284 Statistic J:  0.2806254875692452
mmd_value:  0.007542803883552551 mmd_std:  0.009596030791419853 Statistic J:  0.7860337307688545
mmd_value:  0.00913827121257782 mmd_std:  0.010248371733650107 Statistic J:  0.8916803029863448
mmd_value:  0.010598160326480865 mmd_std:  0.010632384901285253 Statistic J:  0.9967811008421779
mmd_value:  0.011702224612236023 mmd_std:  0.011130422784906177 Statistic J:  1.0513728757999434
mmd_value:  0.012689366936683655 mmd_std:  0.01150231989860624 Statistic J:  1.103200662869866
mmd_value:  0.013490468263626099 mmd_std:  0.01190571359977638 Statistic J:  1.133108750732882
mmd_value:  0.01436874270439148 mmd_std:  0.012150430185641716 Statistic J:  1.1825706978976898
mmd_value:  0.01526351273059845 mmd_std:  0.012496198810228756 Statistic J:  1.221452456254498
mmd_value:  0.01604151725769043 mmd_std:  0.012735449848832549 Statistic J:  1.2595956521442346
tensor([0.3667], device='cuda:0', grad_

testing:   0%|          | 0/100 [00:00<?, ?it/s]

MMD-D: 0
MMD-D: 0
MMD-D: 0
MMD-D: 0
MMD-D: 0
MMD-D: 0
MMD-D: 1
MMD-D: 1
MMD-D: 1
MMD-D: 1
MMD-D: 1
MMD-D: 1
MMD-D: 1
MMD-D: 2
MMD-D: 2
MMD-D: 2
MMD-D: 2
MMD-D: 3
MMD-D: 3
MMD-D: 3
MMD-D: 3
MMD-D: 3
MMD-D: 3
MMD-D: 4
MMD-D: 5
MMD-D: 5
MMD-D: 5
MMD-D: 5
MMD-D: 6
MMD-D: 6
MMD-D: 6
MMD-D: 6
MMD-D: 6
MMD-D: 6
MMD-D: 7
MMD-D: 7
MMD-D: 7
MMD-D: 7
MMD-D: 8
MMD-D: 8
MMD-D: 9
MMD-D: 10
MMD-D: 10
MMD-D: 10
MMD-D: 10
MMD-D: 10
MMD-D: 10
MMD-D: 10
MMD-D: 11
MMD-D: 11
MMD-D: 11
MMD-D: 11
MMD-D: 12
MMD-D: 12
MMD-D: 13
MMD-D: 13
MMD-D: 13
MMD-D: 13
MMD-D: 13
MMD-D: 13
MMD-D: 13
MMD-D: 13
MMD-D: 13
MMD-D: 13
MMD-D: 13
MMD-D: 13
MMD-D: 14
MMD-D: 14
MMD-D: 14
MMD-D: 15
MMD-D: 15
MMD-D: 15
MMD-D: 15
MMD-D: 15
MMD-D: 15
MMD-D: 15
MMD-D: 15
MMD-D: 15
MMD-D: 16
MMD-D: 16
MMD-D: 16
MMD-D: 16
MMD-D: 16
MMD-D: 16
MMD-D: 16
MMD-D: 16
MMD-D: 17
MMD-D: 18
MMD-D: 18
MMD-D: 18
MMD-D: 18
MMD-D: 19
MMD-D: 19
MMD-D: 19
MMD-D: 19
MMD-D: 19
MMD-D: 20
MMD-D: 21
MMD-D: 21
MMD-D: 21
n = 10 --- Test Power of MMD-D:  0.21
n =

testing:   0%|          | 0/100 [00:00<?, ?it/s]

...

MMD-D: 1
MMD-D: 2
MMD-D: 3
MMD-D: 4
MMD-D: 5
MMD-D: 6
MMD-D: 7
MMD-D: 8
MMD-D: 9
MMD-D: 10
MMD-D: 11
MMD-D: 12
MMD-D: 13
MMD-D: 14
MMD-D: 15
MMD-D: 16
MMD-D: 17
MMD-D: 18
MMD-D: 19
MMD-D: 20
MMD-D: 21
MMD-D: 22
MMD-D: 23
MMD-D: 24
MMD-D: 25
MMD-D: 26
MMD-D: 27
MMD-D: 28
MMD-D: 29
MMD-D: 30
MMD-D: 31
MMD-D: 32
MMD-D: 33
MMD-D: 34
MMD-D: 35
MMD-D: 36
MMD-D: 37
MMD-D: 38
MMD-D: 39
MMD-D: 40
MMD-D: 41
MMD-D: 42
MMD-D: 43
MMD-D: 44
MMD-D: 45
MMD-D: 46
MMD-D: 47
MMD-D: 48
MMD-D: 49
MMD-D: 50
MMD-D: 51
MMD-D: 52
MMD-D: 53
MMD-D: 54
MMD-D: 55
MMD-D: 56
MMD-D: 57
MMD-D: 58
MMD-D: 59
MMD-D: 60
MMD-D: 61
MMD-D: 62
MMD-D: 63
MMD-D: 64
MMD-D: 65
MMD-D: 66
MMD-D: 67
MMD-D: 68
MMD-D: 69
MMD-D: 70
MMD-D: 71
MMD-D: 72
MMD-D: 73
MMD-D: 74
MMD-D: 75
MMD-D: 76
MMD-D: 77
MMD-D: 78
MMD-D: 79
MMD-D: 80
MMD-D: 81
MMD-D: 82
MMD-D: 83
MMD-D: 84
MMD-D: 85
MMD-D: 86
MMD-D: 87
MMD-D: 88
MMD-D: 89
MMD-D: 90
MMD-D: 91
MMD-D: 92
MMD-D: 93
MMD-D: 94
MMD-D: 95
MMD-D: 96
MMD-D: 97
MMD-D: 98
MMD-D: 99
MMD-D: 100
n = 100 

testing:   0%|          | 0/100 [00:00<?, ?it/s]

MMD-D: 1
MMD-D: 2
MMD-D: 3
MMD-D: 4
MMD-D: 5
MMD-D: 6
MMD-D: 7
MMD-D: 8
MMD-D: 9
MMD-D: 10
MMD-D: 11
MMD-D: 12
MMD-D: 13
MMD-D: 14
MMD-D: 15
MMD-D: 16
MMD-D: 17
MMD-D: 18
MMD-D: 19
MMD-D: 20
MMD-D: 21
MMD-D: 22
MMD-D: 23
MMD-D: 24
MMD-D: 25
MMD-D: 26
MMD-D: 27
MMD-D: 28
MMD-D: 29
MMD-D: 30
MMD-D: 31
MMD-D: 32
MMD-D: 33
MMD-D: 34
MMD-D: 35
MMD-D: 36
MMD-D: 37
MMD-D: 38
MMD-D: 39
MMD-D: 40
MMD-D: 41
MMD-D: 42
MMD-D: 43
MMD-D: 44
MMD-D: 45
MMD-D: 46
MMD-D: 47
MMD-D: 48
MMD-D: 49
MMD-D: 50
MMD-D: 51
MMD-D: 52
MMD-D: 53
MMD-D: 54
MMD-D: 55
MMD-D: 56
MMD-D: 57
MMD-D: 58
MMD-D: 59
MMD-D: 60
MMD-D: 61
MMD-D: 62
MMD-D: 63
MMD-D: 64
MMD-D: 65
MMD-D: 66
MMD-D: 67
MMD-D: 68
MMD-D: 69
MMD-D: 70
MMD-D: 71
MMD-D: 72
MMD-D: 73
MMD-D: 74
MMD-D: 75
MMD-D: 76
MMD-D: 77
MMD-D: 78
MMD-D: 79
MMD-D: 80
MMD-D: 81
MMD-D: 82
MMD-D: 83
MMD-D: 84
MMD-D: 85
MMD-D: 86
MMD-D: 87
MMD-D: 88
MMD-D: 89
MMD-D: 90
MMD-D: 91
MMD-D: 92
MMD-D: 93
MMD-D: 94
MMD-D: 95
MMD-D: 96
MMD-D: 97
MMD-D: 98
MMD-D: 99
MMD-D: 100
n = 100 

testing:   0%|          | 0/100 [00:00<?, ?it/s]

MMD-D: 1
MMD-D: 2
MMD-D: 3
MMD-D: 4
MMD-D: 5
MMD-D: 6
MMD-D: 7
MMD-D: 8
MMD-D: 9
MMD-D: 10
MMD-D: 11
MMD-D: 12
MMD-D: 13
MMD-D: 14
MMD-D: 15
MMD-D: 16
MMD-D: 17
MMD-D: 18
MMD-D: 19
MMD-D: 20
MMD-D: 21
MMD-D: 22
MMD-D: 23
MMD-D: 24
MMD-D: 25
MMD-D: 26
MMD-D: 27
MMD-D: 28
MMD-D: 29
MMD-D: 30
MMD-D: 31
MMD-D: 32
MMD-D: 33
MMD-D: 34
MMD-D: 35
MMD-D: 36
MMD-D: 37
MMD-D: 38
MMD-D: 39
MMD-D: 40
MMD-D: 41
MMD-D: 42
MMD-D: 43
MMD-D: 44
MMD-D: 45
MMD-D: 46
MMD-D: 47
MMD-D: 48
MMD-D: 49
MMD-D: 50
MMD-D: 51
MMD-D: 52
MMD-D: 53
MMD-D: 54
MMD-D: 55
MMD-D: 56
MMD-D: 57
MMD-D: 58
MMD-D: 59
MMD-D: 60
MMD-D: 61
MMD-D: 62
MMD-D: 63
MMD-D: 64
MMD-D: 65
MMD-D: 66
MMD-D: 67
MMD-D: 68
MMD-D: 69
MMD-D: 70
MMD-D: 71
MMD-D: 72
MMD-D: 73
MMD-D: 74
MMD-D: 75
MMD-D: 76
MMD-D: 77
MMD-D: 78
MMD-D: 79
MMD-D: 80
MMD-D: 81
MMD-D: 82
MMD-D: 83
MMD-D: 84
MMD-D: 85
MMD-D: 86
MMD-D: 87
MMD-D: 88
MMD-D: 89
MMD-D: 90
MMD-D: 91
MMD-D: 92
MMD-D: 93
MMD-D: 94
MMD-D: 95
MMD-D: 96
MMD-D: 97
MMD-D: 98
MMD-D: 99
MMD-D: 100
n = 100 